In [1]:
import cv2
import numpy as np

In [ ]:
## lucas-kanade optical flow
cap = cv2.VideoCapture(0)
ret,frame = cap.read()
prevImg = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
### Shi-Tomashi corner
prevPoints = cv2.goodFeaturesToTrack(prevImg,20, 0.3, 7, 7)
### parameters for lucas kanade optical flow
winSize = (100,100)
maxLevel = 2
criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03)
## mask for drawing purpose
mask = np.zeros_like(frame)
while True:
    ret,frame = cap.read()
    if ret == True:
        nextImg = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        nextPts,status,err = cv2.calcOpticalFlowPyrLK(prevImg,nextImg,prevPoints,None,winSize=winSize,maxLevel=maxLevel,
                                                     criteria=criteria)
        good_prev = prevPoints[status==1]
        good_new = nextPts[status==1]
        
        for i,(new,prev) in enumerate(zip(good_new,good_prev)):
            x_new,y_new = new.ravel()
            x_prev,y_prev = prev.ravel()
            cv2.line(mask,(x_new,y_new),(x_prev,y_prev),(0,255,0),3)
            cv2.circle(frame,(x_new,y_new),5,(255,0,0),-1)
        
        image = cv2.add(mask,frame)
        cv2.imshow('Object Tracking with Lucas_kanade optical flow',image)
        
        prevImg = nextImg.copy()
        prevPoints = good_new.reshape(-1,1,2)
        if cv2.waitKey(3)&0xFF==27:
            break
    else:
        print('capturing Failed!!!')
        break
cap.release()
cv2.destroyAllWindows()

In [2]:
## dense optical flow
cap = cv2.VideoCapture(0)
ret,frame = cap.read()
prevImg = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
hsv_mask = np.zeros_like(frame)
hsv_mask[:,:,1]=255
while True:
    ret,frame2 = cap.read()
    nextImg = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)
    flow = cv2.calcOpticalFlowFarneback(prevImg,nextImg, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    mag, ang = cv2.cartToPolar(flow[:,:,0], flow[:,:,1],angleInDegrees=True)
    hsv_mask[:,:,0] = ang/2
    hsv_mask[:,:,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
    tracking = cv2.cvtColor(hsv_mask,cv2.COLOR_HSV2BGR)
    cv2.imshow('Dense Optical Flow',tracking)
    prevImg = nextImg.copy()
    k = cv2.waitKey(10) & 0xFF
    if k == 27:
        break
    
cap.release()
cv2.destroyAllWindows()
